# TEST OF THE NON-PARALLELIZED ALGORITHM

In [1]:
import torch
import numpy as np
import cvxpy as cp
import time

def knapsack_specialized_pruning_complete(xi, v, w, C):
    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break
            
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1  # Set breakpoints in x_plus vector

    # Edge case: w greater than largest v → allocate entirely to last bucket
    if(w > v[-1]):
        x = torch.zeros(C)
        x[-1] = 1

    # Edge case: w smaller than smallest v → allocate entirely to first bucket
    elif(w < v[0]):
        x = torch.zeros(C)
        x[0] = 1

    # Intermediate case: w within [v[0], v[-1]]
    else:
        # Sort indices by the ratio xi / v
        ratio = (xi / v)
        neg_indices = torch.where(ratio < 0)[0]
        neg_sorted = neg_indices[torch.argsort(ratio[neg_indices], descending=True)]
        pos_indices = torch.where(ratio >= 0)[0]
        pos_sorted = pos_indices[torch.argsort(ratio[pos_indices])]
        b_vector = np.concatenate([neg_sorted, pos_sorted])

        # Find valid first breakpoint with x_plus == 1 and ratio in [0,1]
        b = 0
        while(w / v[b_vector[b]] < 0 or x_plus[b_vector[b]] == 0):
            b += 1

        b_list_new = [b_vector[b]] 

        # Try to find a second valid breakpoint for convex combination
        while(w / v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(w / v[b_vector[b]] < 0):
                continue
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
            if(len(b_list_new) > 2):
                b_list_new.pop(0)

        # If weight ratio exceeds 1, fallback to first index
        if(len(b_list_new) == 1 and w / v[b_list_new[0]] > 1):
            b_list_new[0] = 0

        x = torch.zeros(C)

        # Case: single coefficient solution
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w / v[b_list_new[0]]), 5)

        # Case: convex combination of two coefficients
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 5)
            x[b_list_new[1]] = round(float(1 - theta), 5)

        # Compute objective value
        objective_values = torch.matmul(x, xi)

    # === Compute optimal multipliers ===
    if(x is None):
        lambda_opt = None  # No allocation → no multiplier
    elif(torch.all(x == 0)):
        lambda_opt = 0  # Null allocation → zero multiplier
    elif(torch.count_nonzero(torch.abs(x) > 1e-6) == 2):
        # Two active components → standard finite difference formula
        i = torch.nonzero(torch.abs(x) > 1e-6, as_tuple=True)
        diff = i[0][1] - i[0][0]
        passo = v[1] - v[0]
        lambda_opt = round(float(-1 / (diff * passo) * (xi[i[0][1]] - xi[i[0][0]])), 5)
    elif(torch.count_nonzero(torch.abs(x) > 1e-6) == 1):
        # Single active index → direct multiplier formula
        i = torch.nonzero(torch.abs(x) > 1e-6, as_tuple=True)
        lambda_opt = round(-float(xi[i[0][0]] / v[i[0][0]]), 5)
    else:
        print("ERROR: SOLUTION NOT ACCEPTABLE!")  # Sanity check

    # Final objective value
    objective_values = xi @ x

    return x, lambda_opt, objective_values, x_plus

# === MAIN SCRIPT ===
M = 1 # Number of istances in each batch. M=1 means we are not parallelizing
N = 500 # Number of istances for a certain C

# Buckets are defined in the interval [e0-r, w0+r]
w0 = -0.11
r = 1.1

# Extremes for weights random generation
min_w, max_w = w0 - r, w0 + r

# Threshold to consider a 0
threshold = 1e-2

SUCCESS = 1

start_time = time.time()
for weights_extreme_generation in [0.2, 2]:
    a = -weights_extreme_generation
    b = weights_extreme_generation
    for C in range(5,260, 5):
        print(f"C = {C}, M = {M}, N = {N}, [w0-r, w0+r] = [{w0-r:.2f}, {w0+r:.2f}], [a, b] = [{a}, {b}]")    
        counts_resolvable_instances = 0
        count_solutions_under_threshold = 0
        counts_wrong_multipliers = 0
        counts_wrong_solutions = 0
        for seed in range(N):
            np.random.seed(seed)
            torch.manual_seed(seed)

            xi = torch.sort(torch.rand(C))[0]
            v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
            w = torch.rand(M) * (b - a) + a

            x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)

            # cvxpy solution with inequality constraint (pruning)
            x_pruning = cp.Variable(C)
            equality_constraint_pruning = v @ x_pruning == w
            constraints_pruning = [
                equality_constraint_pruning,
                cp.sum(x_pruning) <= 1,
                x_pruning >= 0,
                x_pruning <= 1
            ]
            objective_pruning = cp.Minimize(xi @ x_pruning)
            prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
            prob_pruning.solve(solver=cp.ECOS)

            threshold = 0.0001
            if(x_pruning.value is not None):
                counts_resolvable_instances += 1
                if(torch.max(x_opt) < threshold):
                    count_solutions_under_threshold += 1
                if(not np.allclose(x_pruning.value, x_opt.numpy(), rtol=1e-5, atol=1e-2)):
                    counts_wrong_solutions += 1
                    SUCCESS = 0
                    print(f"{i} ERROR IN THE SOLUTIONS:")
                if(np.abs(equality_constraint_pruning.dual_value[0] - lambda_opt) > 5e-2):
                    counts_wrong_multipliers += 1
                    SUCCESS = 0
                    print(f"{i} ERROR IN THE MULTIPLIERS:")
                    print(x_pruning.value)
            #else: # to eliminate once verified that the cases of unfeasible solution are correctly mangaed
            #    print("x_opt:", x_opt)
            #    print("x_pruning.value:", x_pruning.value)
            #    print("lambda_opt:", lambda_opt)
            #    print("equality_constraint_pruning.dual_value:", equality_constraint_pruning.dual_value)
            #    print("-"*50)

        print("counts_resolvable_instances:", counts_resolvable_instances)
        print(f"count_solutions_under_threshold ({threshold}):", count_solutions_under_threshold)
        print("counts_wrong_solutions:", counts_wrong_solutions)
        print("counts_wrong_multipliers:", counts_wrong_multipliers)
        print("-"*60)
    print("\n"*5)

training_time = time.time() - start_time
print(f'Time taken by the test: {training_time:.2f} seconds')

if(SUCCESS == 1):
    print("\nThe algorithm is correct!")

C = 5, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]


/opt/anaconda3/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 10, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 1
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 15, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 20, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 1
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 25,

counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 1
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 165, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 170, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 175, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 500
count_solutions_under_threshold (0.0001): 1
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 

counts_resolvable_instances: 253
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 65, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 263
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 70, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 263
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 75, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 266
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 80, M = 1, N = 

counts_resolvable_instances: 272
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 225, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 291
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 230, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 281
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 235, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 280
count_solutions_under_threshold (0.0001): 0
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 240, M = 1, 

# TEST OF THE PARALLELIZED ALGORITHM

In [2]:
def knapsack_specialized_pruning_parallel(xi, v, w, C):
    """
    Solves a specialized knapsack problem using a pruning-based parallel approach.

    Args:
        xi (torch.Tensor): xi variables.
        v (torch.Tensor): Quantization vector.
        w (torch.Tensor): Weight vector.
        C (int): Number of quantization buckets.

    Returns:
        tuple: Optimal allocation (x), optimal multipliers (lambda_opt), objective values, and breakpoint indicator (x_plus).
    """

    M = w.shape[0]

    # Step 1: Compute breakpoint vector x_plus
    b_list = []
    b = 0
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0
        if b != C - 1:
            b_list.append(int(b))
        if b + 1 > C - 1:
            break
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    x_plus[torch.tensor(b_list)] = 1

    # Step 2: Initialize solution and multiplier tensors
    x = torch.zeros(M, C)
    lambda_opt = torch.zeros(M)

    # Step 3: Classify weight vector entries
    v0 = v[0]
    v_last = v[-1]
    mask_small = w < v0
    mask_large = w > v_last
    mask_mid = (~mask_small) & (~mask_large)

    # Case 1: w > v[-1]
    x[mask_large, -1] = 1

    # Case 2: w < v[0]
    x[mask_small, 0] = 1

    # Case 3: Intermediate values v[0] <= w <= v[-1]
    if mask_mid.any():
        M_mid = mask_mid.sum()
        w_mid = w[mask_mid]

        # Sort xi/v ratio to define selection priority
        ratio = xi / v
        neg_indices = torch.where(ratio < 0)[0]
        neg_sorted = neg_indices[torch.argsort(ratio[neg_indices], descending=True)]
        pos_indices = torch.where(ratio >= 0)[0]
        pos_sorted = pos_indices[torch.argsort(ratio[pos_indices])]
        b_vector = torch.cat([neg_sorted, pos_sorted], dim=0)

        # Determine first valid breakpoint index i0
        ratio_b = w_mid[:, None] / v[b_vector]
        x_plus_b = x_plus[b_vector].bool()
        cond1 = (ratio_b >= 0) & x_plus_b
        valid_i0 = cond1.float() * torch.arange(C)[None, :]
        valid_i0[~cond1] = float('inf')
        i0_pos = valid_i0.argmin(dim=1)
        i0 = b_vector[i0_pos]
        v_i0 = v[i0]
        x_single = w_mid / v_i0
        invalid_i0 = x_plus[i0] == 0
        use_two = x_single > 1

        # Determine second index i1 if needed
        i1 = torch.full_like(i0, fill_value=-1)
        if use_two.any():
            b_vector_exp = b_vector.unsqueeze(0).expand(M_mid, -1)
            i0_exp = i0.unsqueeze(1).expand_as(b_vector_exp)
            x_plus_mask = x_plus[b_vector_exp] == 1
            greater_mask = b_vector_exp > i0_exp
            valid_mask = x_plus_mask & greater_mask
            masked_b_vector = torch.where(valid_mask, b_vector_exp, torch.full_like(b_vector_exp, C))
            i1_candidate, _ = masked_b_vector.min(dim=1)
            i0_use_two = i0[use_two]
            i1_candidate_use_two = i1_candidate[use_two]
            valid_i1_use_two = i1_candidate_use_two < C
            i1[use_two] = torch.where(valid_i1_use_two, i1_candidate_use_two, i0_use_two)

        # Step 4: Construct x_mid
        x_mid = torch.zeros(M_mid, C)

        # Subcase 1: Only one active variable
        mask_one = ~use_two
        rows_one = torch.where(mask_one)[0]
        cols_one = i0[mask_one]
        x_mid[rows_one, cols_one] = torch.clamp(torch.round(w_mid[mask_one] / v[cols_one], decimals=5), 0.0, 1.0)

        # Subcase 2: Convex combination of two indices
        mask_two = use_two & (i1 != i0)
        rows_two = torch.where(mask_two)[0]
        idx0 = i0[mask_two]
        idx1 = i1[mask_two]
        v0 = v[idx0]
        v1 = v[idx1]
        w_sel = w_mid[mask_two]
        theta = (w_sel - v1) / (v0 - v1)
        x_mid[rows_two, idx0] = torch.round(theta, decimals=5)
        x_mid[rows_two, idx1] = torch.round(1 - theta, decimals=5)

        # Assign computed x_mid to global solution
        x[mask_mid] = x_mid

    # Step 5: Compute optimal multipliers (lambda_opt)
    eps = 1e-6
    nz_mask = torch.abs(x) > eps
    nz_counts = nz_mask.sum(dim=1)

    # Case: one active variable
    m1 = torch.where(nz_counts == 1)[0]
    if m1.numel() > 0:
        submask = nz_mask[m1]
        indices = submask.nonzero(as_tuple=False)
        i = indices[:, 1]
        lambda_opt[m1] = -xi[i] / v[i]
        lambda_opt[m1] = torch.round(lambda_opt[m1], decimals=5)

    # Case: two active variables
    m2 = torch.where(nz_counts == 2)[0]
    if m2.numel() > 0:
        indices = nz_mask[m2].nonzero().reshape(-1, 2)
        grouped = indices.view(-1, 2, 2)
        i = grouped[:, 0, 1]
        j = grouped[:, 1, 1]
        delta_xi = xi[j] - xi[i]
        delta_idx = j - i
        passo = v[1] - v[0]
        lambda_opt[m2] = -delta_xi / (delta_idx * passo)
        lambda_opt[m2] = torch.round(lambda_opt[m2], decimals=5)

    # Step 6: Compute objective values
    objective_values = x @ xi

    return x, lambda_opt, objective_values, x_plus

# === MAIN SCRIPT ===
M = 100 # Number of istances in each batch
N = 500 # Number of istances for a certain C

# Buckets are defined in the interval [e0-r, w0+r]
w0 = -0.11
r = 1.1

# Extremes for weights random generation
min_w, max_w = w0 - r, w0 + r

# Threshold to consider a 0
threshold = 1e-2

SUCCESS = 1

start_time = time.time()
for weights_extreme_generation in [0.2, 2]:
    a = -weights_extreme_generation
    b = weights_extreme_generation
    for C in range(5, 260, 5):
        print(f"C = {C}, M = {M}, N = {N}, [w0-r, w0+r] = [{w0-r:.2f}, {w0+r:.2f}], [a, b] = [{a}, {b}]")    
        counts_resolvable_instances = 0
        count_solutions_under_threshold = 0
        counts_wrong_multipliers = 0
        counts_wrong_solutions = 0
        for seed in range(N):
            np.random.seed(seed)
            torch.manual_seed(seed)

            xi_all = torch.sort(torch.rand(C))[0]
            v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
            w_all = torch.rand(M) * (b - a) + a

            x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi_all, v, w_all, C)

            for i in range(M):
                w = w_all[i].unsqueeze(0)
                x_opt2, lambda_opt2, phi_opt2, x_plus2 = knapsack_specialized_pruning_complete(xi_all, v, w, C)

                if x_opt2 is not None:
                    counts_resolvable_instances += 1
                    if torch.max(x_opt[i]) < threshold:
                        count_solutions_under_threshold += 1
                    if(not np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2)):                    
                        counts_wrong_solutions += 1
                        SUCCESS = 0
                    if lambda_opt2 is not None and abs(lambda_opt2 - lambda_opt[i]) > 1e-3:
                        counts_wrong_multipliers += 1
                        SUCCESS = 0

        print("counts_resolvable_instances:", counts_resolvable_instances)
        print(f"count_solutions_under_threshold ({threshold}):", count_solutions_under_threshold)
        print("counts_wrong_solutions:", counts_wrong_solutions)      
        print("counts_wrong_multipliers:", counts_wrong_multipliers)
        print("-"*60)
    print("\n"*5)

training_time = time.time() - start_time
print(f'Time taken by the test: {training_time:.2f} seconds')

C = 5, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2195
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 10, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2510
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 15, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2570
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 20, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2643
counts_wrong_solutions: 0
co

counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2765
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 160, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2775
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 165, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2765
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
------------------------------------------------------------
C = 170, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 2776
counts_wrong_solutions: 0
counts_wrong_multipliers: 0
-----------------------------------------------

counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 309
counts_wrong_solutions: 5
counts_wrong_multipliers: 5
------------------------------------------------------------
C = 60, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 310
counts_wrong_solutions: 2
counts_wrong_multipliers: 2
------------------------------------------------------------
C = 65, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 304
counts_wrong_solutions: 7
counts_wrong_multipliers: 7
------------------------------------------------------------
C = 70, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 301
counts_wrong_solutions: 5
counts_wrong_multipliers: 5
------------------------------------------------------------
C = 7

counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 238
counts_wrong_solutions: 2
counts_wrong_multipliers: 2
------------------------------------------------------------
C = 215, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 239
counts_wrong_solutions: 2
counts_wrong_multipliers: 2
------------------------------------------------------------
C = 220, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 243
counts_wrong_solutions: 3
counts_wrong_multipliers: 3
------------------------------------------------------------
C = 225, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 242
counts_wrong_solutions: 3
counts_wrong_multipliers: 3
------------------------------------------------------------
C 

In [3]:
#133 errors over 5100000

# [DEBUG] Study of instance subgroups to understand what is going wrong

In [4]:
# === MAIN SCRIPT ===
M = 100 # Number of istances in each batch
N = 500 # Number of istances for a certain C

# Buckets are defined in the interval [e0-r, w0+r]
w0 = -0.11
r = 1.1

# Extremes for weights random generation
min_w, max_w = w0 - r, w0 + r

# Threshold to consider a 0
threshold = 1e-2

SUCCESS = 1

for weights_extreme_generation in [2]:
    a = -weights_extreme_generation
    b = weights_extreme_generation
    for C in [20]:
        print(f"C = {C}, M = {M}, N = {N}, [w0-r, w0+r] = [{w0-r:.2f}, {w0+r:.2f}], [a, b] = [{a}, {b}]")    
        counts_resolvable_instances = 0
        count_solutions_under_threshold = 0
        counts_wrong_multipliers = 0
        counts_wrong_solutions = 0
        for seed in range(N):
            np.random.seed(seed)
            torch.manual_seed(seed)

            xi_all = torch.sort(torch.rand(C))[0]
            v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
            w_all = torch.rand(M) * (b - a) + a

            x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi_all, v, w_all, C)

            for i in range(M):
                w = w_all[i].unsqueeze(0)
                x_opt2, lambda_opt2, phi_opt2, x_plus2 = knapsack_specialized_pruning_complete(xi_all, v, w, C)
                if x_opt2 is not None:
                    counts_resolvable_instances += 1
                    if torch.max(x_opt[i]) < threshold:
                        count_solutions_under_threshold += 1
                    if(not np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2)):                   
                        counts_wrong_solutions += 1
                        SUCCESS = 0
                    if lambda_opt2 is not None and abs(lambda_opt2 - lambda_opt[i]) > 1e-3:
                        print(f"\n\n\n{seed},{i} ERROR IN THE MULTIPLIERS:")
                        print("xi_all:", xi_all)
                        print("v:", v)
                        print("w:", w)
                        print("x_opt[i]:", x_opt[i])
                        print("x_opt2:", x_opt2)
                        print("lambda_opt[i]:", lambda_opt[i])
                        print("lambda_opt2:", lambda_opt2)                        
                        counts_wrong_multipliers += 1
                        SUCCESS = 0

        print("\n\n\ncounts_resolvable_instances:", counts_resolvable_instances)
        print(f"count_solutions_under_threshold ({threshold}):", count_solutions_under_threshold)
        print("counts_wrong_solutions:", counts_wrong_solutions)
        print("counts_wrong_multipliers:", counts_wrong_multipliers)
        print("-"*60)

C = 20, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]



226,3 ERROR IN THE MULTIPLIERS:
xi_all: tensor([0.0591, 0.0626, 0.0774, 0.1073, 0.1091, 0.1205, 0.2461, 0.2757, 0.2964,
        0.3749, 0.3846, 0.3888, 0.3892, 0.4415, 0.4861, 0.4864, 0.6760, 0.7381,
        0.9579, 0.9993])
v: tensor([-1.2100e+00, -1.1000e+00, -9.9000e-01, -8.8000e-01, -7.7000e-01,
        -6.6000e-01, -5.5000e-01, -4.4000e-01, -3.3000e-01, -2.2000e-01,
        -1.1000e-01, -5.9605e-08,  1.1000e-01,  2.2000e-01,  3.3000e-01,
         4.4000e-01,  5.5000e-01,  6.6000e-01,  7.7000e-01,  8.8000e-01])
w: tensor([0.8049])
x_opt[i]: tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.6587,
         0.0000,  1.6587,  0.0000,  0.0000])
x_opt2: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3413,





counts_resolvable_instances: 50000
count_solutions_under_threshold (0.01): 282
counts_wrong_solutions: 12
counts_wrong_multipliers: 12
------------------------------------------------------------


# TIME COMPARISON BETWEEN THE NON-PARALLELIZED ALGORITHM AND CVXPY​

In [5]:
C = 24
M = 1

w0 = -0.01
r = 1.32
a = -2
b = 2.2
min_w, max_w = w0 - r, w0 + r

np.random.seed(41)  
torch.manual_seed(41)

print("Start Specialized Algorithm")
start_time = time.time()
for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M) * (b - a) + a
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)
training_time = time.time() - start_time
print(f'Time taken by the specialized algorithm: {training_time:.2f} seconds')

print("Start cvxpy")
start_time = time.time()
for i in range(1000):
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
training_time2 = time.time() - start_time
print(f'Time taken by cvxpy: {training_time2:.2f} seconds')

print(f"\nThe Specialized Algorithm is {training_time2/training_time:.2f}X more perfomant than cvxpy")

Start Specialized Algorithm
Time taken by the specialized algorithm: 0.17 seconds
Start cvxpy
Time taken by cvxpy: 1.61 seconds

The Specialized Algorithm is 9.61X more perfomant than cvxpy


# TIME COMPARISON BETWEEN THE PARALLELIZED ALGORITHM AND CVXPY

In [ ]:
C = 32
M = 44000
N = 10

w0 = -0.01
r = 1.32
a = -2
b = 2.2
min_w, max_w = w0 - r, w0 + r

np.random.seed(41)  
torch.manual_seed(41)

print("Start Specialized Algorithm")
start_time = time.time()
for i in range(N):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M) * (b - a) + a
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi, v, w, C)
training_time = time.time() - start_time
print(f'Time taken by the specialized algorithm: {training_time:.2f} seconds')

print("Start cvxpy")
start_time = time.time()
for j in range(N):
    for i in range(M):
        # cvxpy solution with inequality constraint (pruning)
        x_pruning = cp.Variable(C)
        equality_constraint_pruning = v @ x_pruning == w[i]
        constraints_pruning = [
            equality_constraint_pruning,
            cp.sum(x_pruning) <= 1,
            x_pruning >= 0,
            x_pruning <= 1
        ]
        objective_pruning = cp.Minimize(xi @ x_pruning)
        prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
        prob_pruning.solve(solver=cp.SCS)
training_time2 = time.time() - start_time
print(f'Time taken by cvxpy: {training_time2:.2f} seconds')

print(f"\nThe Specialized Algorithm is {training_time2/training_time:.2f}X more perfomant than cvxpy")

Start Specialized Algorithm
Time taken by the specialized algorithm: 0.08 seconds
Start cvxpy


/opt/anaconda3/lib/python3.11/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
